In [ ]:
!pip install -q sentence-transformers scikit-learn


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
ma = {
    "Anxiety Disorders": ["Cognitive Behavioral Therapy (CBT)", "Mindfulness Therapy"],
    "Depression": ["CBT", "Interpersonal Therapy"],
    "Bipolar Disorder": ["Medication Management", "CBT"],
    "Schizophrenia": ["Antipsychotic Medication", "Supportive Therapy"],
    "Obsessive-Compulsive Disorder (OCD)": ["Exposure Therapy", "CBT"],
    "Post-Traumatic Stress Disorder (PTSD)": ["Trauma-Focused CBT", "EMDR"],
    "Autism Spectrum Disorder": ["Behavioral Therapy", "Speech Therapy"],
    "ADHD": ["Behavioral Therapy", "Medication Management"],
    "Eating Disorders": ["Nutritional Counseling", "CBT"],
    "Substance Use Disorders": ["Motivational Interviewing", "Detox + Rehab"],
    "Grief / Loss": ["Grief Counseling", "Supportive Therapy"],
    "Stress / Burnout": ["Mindfulness Therapy", "CBT"],
    "Relationship Issues": ["Couples Therapy", "Supportive Therapy"],
    "Self-Harm / Suicidal Thoughts": ["Crisis Intervention", "CBT"],
    "Anger Issues": ["Anger Management Therapy", "CBT"],
    "Sleep Disorders / Insomnia": ["Sleep Hygiene Training", "CBT-I"],
    "Personality Disorders": ["Dialectical Behavior Therapy (DBT)", "Schema Therapy"],
    "Phobias": ["Exposure Therapy", "CBT"],
    "Body Dysmorphic Disorder": ["CBT", "Mindfulness Therapy"],
    "Social Anxiety Disorder": ["CBT", "Group Therapy"],
    "Sexual Health Concerns": ["Sex Therapy", "CBT"],
    "Workplace Stress": ["Career Counseling", "Mindfulness Therapy"],
    "Parenting Challenges": ["Family Therapy", "Parent Coaching"],
    "Identity / Gender Dysphoria": ["Affirmative Therapy", "Gender Counseling"],
    "Loneliness / Isolation": ["Support Groups", "CBT"]
}

In [4]:
def generate_examples(ma):
    examples = []
    for condition in ma:
        phrases = [
            f"I've been dealing with {condition.lower()}.",
            f"I feel overwhelmed due to {condition.lower()}",
            f"Symptoms point to {condition.lower()}",
            f"Need help managing {condition.lower()}",
            f"I might be facing {condition.lower()}, what should I do?"
        ]
        for p in phrases:
            examples.append({"text": p, "label": condition})
    return examples

In [5]:
data = generate_examples(ma)
texts = [d["text"] for d in data]
labels = [d["label"] for d in data]

In [6]:
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

In [8]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
X = embedder.encode(texts)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [11]:
clf = LogisticRegression(max_iter=2000, random_state=42)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=2000, random_state=42)

In [13]:
y_pred = clf.predict(X_test)
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


 Classification Report:

                                       precision    recall  f1-score   support

                                 ADHD       1.00      1.00      1.00         1
                         Anger Issues       1.00      1.00      1.00         1
                    Anxiety Disorders       1.00      1.00      1.00         1
             Autism Spectrum Disorder       1.00      1.00      1.00         1
                     Bipolar Disorder       1.00      1.00      1.00         1
             Body Dysmorphic Disorder       1.00      1.00      1.00         1
                           Depression       1.00      1.00      1.00         1
                     Eating Disorders       1.00      1.00      1.00         1
                         Grief / Loss       1.00      1.00      1.00         1
          Identity / Gender Dysphoria       1.00      1.00      1.00         1
               Loneliness / Isolation       1.00      1.00      1.00         1
  Obsessive-Compulsive Di

In [14]:
def predict_user_query(query):
    vec = embedder.encode([query])
    pred = clf.predict(vec)[0]
    label = encoder.inverse_transform([pred])[0]
    therapy = ma[label]
    return {
        "Predicted Category": label,
        "Suggested Therapies": therapy
    }

In [15]:
while True:
    query = input(" Describe your mental health concern (or type 'exit' to stop):\n> ")
    if query.lower() == 'exit':
        break
    result = predict_user_query(query)
    print("\n Predicted Condition:", result["Predicted Category"])
    print(" Suggested Therapies:", ", ".join(result["Suggested Therapies"]))
    print("-" * 50)

 Describe your mental health concern (or type 'exit' to stop):
> I tend to scratch myself whenever somone tries to talk to me

 Predicted Condition: Phobias
 Suggested Therapies: Exposure Therapy, CBT
--------------------------------------------------
 Describe your mental health concern (or type 'exit' to stop):
> exit
